In [1]:
import pandas as pd
import pickle
import time

X_path = "/kaggle/input/pawpularity-features-pickle/X_features.pkl"
debut = time.time()
df = pickle.load(open(X_path, 'rb'))
print("Temps écoulé : {:.2f}".format(time.time() - debut))
df

Temps écoulé : 47.73


,0,1,2,3,4,5,6,7,8,9,...,62722,62723,62724,62725,62726,62727,62728,62729,62730,62731
0,0,1,1,1,0,0,1,0,0,0,...,-1.688457e-11,-0.002236,-0.019697,6.138978,37.496304,-7.124449e-10,2.135291,-0.000004,-1.635739e-14,-0.159953
1,0,1,1,0,0,0,0,0,0,0,...,-1.468612e-11,-0.000810,-0.009978,6.797769,36.680656,-4.828979e-10,1.992804,-0.000002,-1.420318e-14,-0.120112
2,0,1,1,1,0,0,0,0,1,1,...,-9.352968e-11,-0.004833,-0.020676,7.370453,39.557130,-1.420247e-09,2.103800,-0.000012,-2.998134e-14,-0.196348
3,0,1,1,1,0,0,0,0,0,0,...,-3.073597e-11,-0.002168,-0.016478,6.751638,38.550850,-7.190994e-10,2.122260,-0.000005,-1.738771e-14,-0.168751
4,0,0,0,1,0,0,1,0,0,0,...,-1.126881e-11,-0.002349,-0.015439,5.904394,37.275562,-4.514170e-10,2.095064,-0.000005,-3.069000e-14,-0.153185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9939,0,1,1,1,0,0,0,0,0,0,...,-1.218977e-10,-0.005017,-0.027896,7.828807,39.470863,-1.347306e-09,2.221529,-0.000014,-4.130087e-14,-0.208505
9940,0,1,1,0,0,0,0,0,0,0,...,-1.308031e-11,-0.002139,-0.014797,6.319387,38.301490,-6.742156e-10,1.845331,-0.000004,-1.582974e-14,-0.139827
9941,0,0,1,1,0,0,0,0,0,0,...,-5.323584e-11,-0.002414,-0.022607,7.020040,38.955620,-1.417024e-09,1.985678,-0.000006,-1.653711e-14,-0.161103
9942,0,1,1,1,0,0,0,0,0,0,...,-1.190707e-12,-0.000292,-0.002824,6.478369,32.678135,-2.304934e-11,2.553012,-0.000001,-7.332549e-14,-0.072156


In [2]:
import numpy as np

# En expérimentant, réaliser une matrice de corrélation sur 5000 colonnes
# est faisable en 10mn environ : le mieux est d'itérer sur l'ensemble des
# colonnes pour supprimer les corrélations par "lot"

step = 5000
correlated_features = []
for i in range(13):
    debut = time.time()
    sub_df = df[df.columns[i * step:][:step]]
    correlation_matrix = sub_df.corr()
    print("> Matrice de corrélation", i, "calculée")
    print("  Temps écoulé : {:.2f}".format(time.time() - debut))
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
    for column in upper.columns:
        if any(upper[column] > 0.8):
            correlated_features.append(column)
    print("  Nb colonnes à supprimer :", len(correlated_features))

df.drop(labels=correlated_features, axis=1, inplace=True)
df.to_pickle("X_features_cleaned.pkl")
pickle.dump(correlated_features, open("correlated_features.pkl", "wb"))

> Matrice de corrélation 0 calculée
  Temps écoulé : 667.29
  Nb colonnes à supprimer : 2322
> Matrice de corrélation 1 calculée
  Temps écoulé : 663.78
  Nb colonnes à supprimer : 4936
> Matrice de corrélation 2 calculée
  Temps écoulé : 661.42
  Nb colonnes à supprimer : 6798
> Matrice de corrélation 3 calculée
  Temps écoulé : 667.47
  Nb colonnes à supprimer : 8894
> Matrice de corrélation 4 calculée
  Temps écoulé : 662.43
  Nb colonnes à supprimer : 10414
> Matrice de corrélation 5 calculée
  Temps écoulé : 661.63
  Nb colonnes à supprimer : 11935
> Matrice de corrélation 6 calculée
  Temps écoulé : 661.90
  Nb colonnes à supprimer : 13229
> Matrice de corrélation 7 calculée
  Temps écoulé : 661.93
  Nb colonnes à supprimer : 14777
> Matrice de corrélation 8 calculée
  Temps écoulé : 663.62
  Nb colonnes à supprimer : 16250
> Matrice de corrélation 9 calculée
  Temps écoulé : 661.68
  Nb colonnes à supprimer : 18346
> Matrice de corrélation 10 calculée
  Temps écoulé : 662.21
  N

In [3]:
df.shape

(9944, 37885)

In [4]:
correlated_features_2 = []
for i in range(round(df.shape[1] / step)):
    debut = time.time()
    sub_df = df[df.columns[i * step:][:step]]
    correlation_matrix = sub_df.corr()
    print("> Matrice de corrélation", i, "calculée")
    print("  Temps écoulé : {:.2f}".format(time.time() - debut))
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
    for column in upper.columns:
        if any(upper[column] > 0.8):
            correlated_features_2.append(column)
    print("  Nb colonnes à supprimer :", len(correlated_features_2))

df.drop(labels=correlated_features_2, axis=1, inplace=True)
df.to_pickle("X_features_cleaned_2.pkl")
pickle.dump(correlated_features_2, open("correlated_features_2.pkl", "wb"))

> Matrice de corrélation 0 calculée
  Temps écoulé : 667.37
  Nb colonnes à supprimer : 217
> Matrice de corrélation 1 calculée
  Temps écoulé : 662.95
  Nb colonnes à supprimer : 298
> Matrice de corrélation 2 calculée
  Temps écoulé : 668.06
  Nb colonnes à supprimer : 411
> Matrice de corrélation 3 calculée
  Temps écoulé : 663.28
  Nb colonnes à supprimer : 594
> Matrice de corrélation 4 calculée
  Temps écoulé : 674.80
  Nb colonnes à supprimer : 630
> Matrice de corrélation 5 calculée
  Temps écoulé : 673.51
  Nb colonnes à supprimer : 688
> Matrice de corrélation 6 calculée
  Temps écoulé : 672.13
  Nb colonnes à supprimer : 778
> Matrice de corrélation 7 calculée
  Temps écoulé : 221.86
  Nb colonnes à supprimer : 792


In [5]:
df.shape

(9944, 37093)

In [6]:
y_path = "/kaggle/input/pawpularity-features-targets/y_targets.csv"
df_target = pd.read_csv(y_path, header=None)
df_target.shape

(9944, 1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df_target, test_size=0.2, random_state=42)

In [8]:
# free memory
import os, psutil, gc
def usage():
    process = psutil.Process(os.getpid())
    print(process.memory_info()[0] / float(2 ** 20))
usage()
del df
del df_target
gc.collect()
usage()

6238.28515625
6238.28515625


In [9]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(7955, 37093) (1989, 37093) (7955, 1) (1989, 1)


In [10]:
import xgboost as xgb
xgb_regressor = xgb.XGBRegressor(random_state=123)
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
pickle.dump(xgb_regressor, open("xgb_regressor.pkl", "wb"))

In [12]:
y_pred = xgb_regressor.predict(X_test)

print("mean_absolute_error : {:.2f}".format(mean_absolute_error(y_test, y_pred)))
mse = mean_squared_error(y_test, y_pred)
print("mean_squared_error : {:.2f}".format(mse))
print("root_mean_squared_error : {:.2f}".format(np.sqrt(mse)))
print("rmpse : {:.2f}".format(np.sqrt(np.mean(np.square(((y_test - y_pred) / y_test))))))
print("mean_absolute_percentage_error : {:.2f}".format(mean_absolute_percentage_error(y_test, y_pred)))
print("r2_score : {:.2f}".format(r2_score(y_test, y_pred)))

NameError: name 'mean_absolute_error' is not defined

In [ ]:
import os
from PIL import Image

test_dir = '/kaggle/input/petfinder-pawpularity-score/test'

test_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv", sep=',')
test_df['Id'] = test_df['Id'] + '.jpg'
test_df = test_df.rename(columns={'Id': 'filename'})
print("shape test_df: ", test_df.shape)

width, height = 224, 224
batch_size = 32

import sys
import os
sys.path.insert(0, "/kaggle/input/efnetv2src/efficientnet-v2-keras-main")
sys.path.append('../input/tfkeras-efficientnetsv2/')

from efficientnet_v2 import EfficientNetV2XL
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.metrics import MeanAbsoluteError, MeanAbsolutePercentageError
import tensorflow_addons as tfa

efficientnet = EfficientNetV2XL(
    include_top=False,
    weights='../input/tfkeras-efficientnetsv2/21_ft1k_notop/efficientnetv2-xl-21k-ft1k_notop.h5', 
    input_shape=(height, width, 3)
)

efficientnet.trainable = False

radam = tfa.optimizers.RectifiedAdam(learning_rate=0.001)
optimizer = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

efficientnet.compile(
    optimizer=optimizer,
    loss="mean_absolute_error",
    metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()]
)

import tensorflow as tf
def preprocess(image):  
    return (tf.cast(image, dtype=tf.float32) - 128.00) / 128.00

ids = []
pawpularities = []
for test_image in os.listdir(test_dir):
    image_path = os.path.join(test_dir, test_image)
    id_image = test_image.split('.')[0]
    ids.append(id_image)
    img = Image.open(image_path) 
    img = img.resize((width, height))
    img = preprocess(np.array(img).reshape(1, width, height, 3))
    # on fait la prediction avec EfficientNet
    img_features = efficientnet.predict(img)
    features_vec = ",".join([str(v) for v in img_features.flatten()])
    # on supprime les colonnes corrélées
    features_vec = np.delete(features_vec, correlated_features)
    features_vec = np.delete(features_vec, correlated_features_2)
    # on récupère les colonnes existantes
    existing_cols = test_df[test_df['filename'] == test_image]
    existing_cols = existing_cols.drop(labels=['filename'], axis=1)
    existing_cols = existing_cols.to_csv(header=False, index=False).rstrip()
    X_pred = []
    X_pred.append("{},{}\n".format(existing_cols, features_vec))
    # on fait la prediction finale
    y_pred = xgb_regressor.predict(X_pred)
    # on ajoute ça à la liste des prédictions
    pawpularities.append(y_pred[0])

submission_dict = {
    'Id': ids,
    'Pawpularity': pawpularities
}
submission_df = pd.DataFrame(submission_dict)
print(submission_df)
submission_df.to_csv('submission.csv', index=False, sep=',')